In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
import time
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torchsummary import summary

In [2]:
data_dir = './data/train'
def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([transforms.Resize(200),
                                       transforms.ToTensor(),
                                       ])    
    test_transforms = transforms.Compose([transforms.Resize(200),
                                      transforms.ToTensor(),
                                      ])    
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)    
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=8)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=8)
    return trainloader, testloader 

trainloader,testloader = load_split_train_test(data_dir, .2)
print(trainloader.dataset.classes)

['with_mask', 'without_mask']


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")

In [4]:
model = models.mobilenet_v3_large(pretrained = True)
summary(model, (3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 100, 100]             432
       BatchNorm2d-2         [-1, 16, 100, 100]              32
         Hardswish-3         [-1, 16, 100, 100]               0
            Conv2d-4         [-1, 16, 100, 100]             144
       BatchNorm2d-5         [-1, 16, 100, 100]              32
              ReLU-6         [-1, 16, 100, 100]               0
            Conv2d-7         [-1, 16, 100, 100]             256
       BatchNorm2d-8         [-1, 16, 100, 100]              32
          Identity-9         [-1, 16, 100, 100]               0
 InvertedResidual-10         [-1, 16, 100, 100]               0
           Conv2d-11         [-1, 64, 100, 100]           1,024
      BatchNorm2d-12         [-1, 64, 100, 100]             128
             ReLU-13         [-1, 64, 100, 100]               0
           Conv2d-14           [-1, 64,

In [5]:
model.classifier

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=1000, bias=True)
)

In [6]:
model.classifier = nn.Sequential(
  nn.Linear(in_features=960, out_features=1280, bias=True),
  nn.Hardswish(),
  nn.Dropout(p=0.2, inplace=True),
  nn.Linear(in_features=1280, out_features=1000, bias=True),
  nn.Linear(in_features=1000, out_features=2, bias=True),
)

In [7]:
for param in model.parameters():
    param.requires_grad = True

In [8]:
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
model.to(device)

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): Identity()
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=

In [9]:
epochs = 2
steps = 0
epochs_no_improve=0
n_epochs_stop = 6
running_loss = 0
print_every = 100
min_val_loss = np.Inf
train_losses, test_losses = [], []
for epoch in range(epochs):
    start = time.time()
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model(inputs)  
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    print(1)
    test_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device),labels.to(device)
            logps = model(inputs)
            batch_loss = criterion(logps, labels)
            test_loss += batch_loss.item()
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy +=torch.mean(equals.type(torch.FloatTensor)).item()
    train_losses.append(running_loss/len(trainloader))
    test_losses.append(test_loss/len(testloader))
    test_accuracy=round(accuracy/len(testloader,4))
    epoch_losses =  np.mean(test_loss)
    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {running_loss/print_every:.3f}.. "
          f"Test loss: {test_loss/len(testloader):.3f}.. "
          f"Test accuracy: {accuracy/len(testloader):.3f}")
    running_loss = 0
    model.train()
    end = time.time()
    print("** Epoch {} ** - Epoch Time {}".format(epoch, int(end-start)))
    if epoch_losses < min_val_loss:
        epochs_no_improve = 0
        min_val_loss = epoch_losses
        torch.save(model, 'best_model.pth')                
    else:
        print(2)
        epochs_no_improve += 1
    if epoch > 5 and epochs_no_improve == n_epochs_stop:
        print('Early stopping!' )
        early_stop = True
    if early_stop:
        print('stoppend')
        break
torch.save(model, 'aerialmodel.pth')

KeyboardInterrupt: 

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

In [ ]:
test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                     ])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1=torch.load('aerialmodel.pth')
model1.eval()

In [ ]:
def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model1(input)
    index = output.data.cpu().numpy().argmax()
    return index